In [ ]:
#Script to get average speaking times per speaker as well as ratios
import os
import pandas as pd

In [ ]:
paths = ['~/WavFiles_extraAudioAnalysis/transcripts', '~/WavFiles_extraAudioAnalysis_batch2/transcripts', '~/WavFiles_extraAudioAnalysis_batch3/transcripts']

data = []

In [ ]:
def collect_files(base_path):
    all_files = []
    for root, dirs, files in os.walk(base_path):
        all_files.extend([os.path.join(root, file) for file in files])
    return all_files

In [ ]:
def process_directory(directory):
    for filename in os.listdir(directory):
    #for filename in directory:
        if filename.endswith('.csv'):
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath, delimiter=';')
            #df = pd.read_csv(filename, delimiter=';')
            # Count sentences for each speaker
            speaker1_count = df[df['speaker'] == 'SPEAKER_00'].shape[0]
            speaker2_count = df[df['speaker'] == 'SPEAKER_01'].shape[0]
            total_sentences = speaker1_count + speaker2_count
            
            # Ensure we avoid division by zero
            if total_sentences == 0:
                ratio = float('inf')
            else:
                ratio = abs((speaker1_count / total_sentences) - (speaker2_count / total_sentences))
            
            data.append({
                'transcript_id': filename,
                'speaker1_sentences': speaker1_count,
                'speaker2_sentences': speaker2_count,
                'ratio': ratio
            })


In [ ]:
for path in paths:
    #all_files = collect_files(path)
    process_directory(path)

df = pd.DataFrame(data)

In [ ]:
df.shape

In [ ]:
print(df['speaker2_sentences'].unique())

In [ ]:
average_ratio = df[df['ratio'] != float('inf')]['ratio'].mean()
print(f"Average Ratio: {average_ratio}")

In [ ]:
threshold = 1.25 * average_ratio
print(f"Threshold: {threshold}")

In [ ]:
average_filtered = df[(df['ratio'] <= average_ratio) & (df['ratio'] != float('inf'))]

print("Filtered Transcripts:")
print(average_filtered.shape[0])
print(df.shape[0])

In [ ]:
threshold_filtered = df[(df['ratio'] <= threshold) & (df['ratio'] != float('inf'))]

print("Filtered Transcripts:")
print(threshold_filtered.shape[0])
print(df.shape[0])

In [ ]:
average_filtered.to_csv('~/WavFiles_extraAudioAnalysis_batch3/Average_filtered_dataframe.csv', index=False)

In [ ]:
threshold_filtered.to_csv('~/WavFiles_extraAudioAnalysis_batch3/threshold_filtered_dataframe.csv', index=False)

In [ ]:
df.to_csv('~/WavFiles_extraAudioAnalysis_batch3/all_ratios.csv', index=False)